In [2]:
from glob import glob
from tqdm import tqdm
import pickle
import numpy as np

### Define paths

In [3]:
import platform
import os

if platform.system() == 'Darwin':
    DATA_PATH = "/Users/maltegenschow/Documents/Uni/Thesis/Data.nosync"
    ROOT_PATH = "/Users/maltegenschow/Documents/Uni/Thesis/Thesis"
elif platform.system() == 'Linux':
    DATA_PATH = "/pfs/work7/workspace/scratch/tu_zxmav84-thesis/Data.nosync"
    ROOT_PATH = "/pfs/work7/workspace/scratch/tu_zxmav84-thesis/Thesis"

current_wd = os.getcwd()

In [4]:
real_path = f"{DATA_PATH}/Zalando_Germany_Dataset/dresses/images/e4e_images/all/"
generated_path_00003 = f"{DATA_PATH}/Generated_Images/e4e/00003_snapshot_920/"
generate_path_00005 = f"{DATA_PATH}/Generated_Images/e4e/00005_snapshot_1200/"

scores_save_path_00003 = f"{DATA_PATH}/Metrics/LPIPS/00003_snapshot_920/lpips_scores_00003_snapshot_920"
scores_save_path_00005 = f"{DATA_PATH}/Metrics/LPIPS/00005_snapshot_1200/lpips_scores_00005_snapshot_1200"

### Setup LPIPS

In [5]:
import lpips
import matplotlib.pyplot as plt
loss_fn_alex = lpips.LPIPS(net='alex') # best forward scores
loss_fn_alex = loss_fn_alex.to('cuda')

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /pfs/work7/workspace/scratch/tu_zxmav84-thesis/miniconda3/envs/thesis/lib/python3.7/site-packages/lpips/weights/v0.1/alex.pth


In [6]:
def get_lpips(sku, generated_path, real_path):
    real = lpips.im2tensor(lpips.load_image(f"{real_path}{sku}"))
    fake = lpips.im2tensor(lpips.load_image(f"{generated_path}{sku}"))
    fake = fake.cuda()
    real = real.cuda()
    score = loss_fn_alex.forward(real, fake)
    return score.item()

In [7]:
skus = glob(f"{real_path}*.jpg")
skus = [elem.split('/')[-1] for elem in skus]

### Calculate LPIPS Scores for e4e from 00003_snapshot_920

In [8]:
if not os.path.exists(scores_save_path_00003):
    lpips_scores_00003 = {sku: None for sku in skus}
    for sku in tqdm(skus):
        score = get_lpips(sku, generated_path_00003, real_path)
        lpips_scores_00003[sku] = score

    with open(scores_save_path_00003, 'wb') as handle:
        pickle.dump(lpips_scores_00003, handle, protocol=pickle.HIGHEST_PROTOCOL)
else: 
    with open(scores_save_path_00003, 'rb') as f:
        lpips_scores_00003 = pickle.load(f)

In [9]:
print(f"Mean LPIPS scores: {np.mean(list(lpips_scores_00003.values()))}")

Mean LPIPS scores: 0.14481622185478593


In [26]:
# Differentiate in Training and Test set
train_skus = glob(f"{DATA_PATH}/Zalando_Germany_Dataset/dresses/images/e4e_images/train/*.jpg")
train_skus = [elem.split('/')[-1] for elem in train_skus]
test_skus = glob(f"{DATA_PATH}/Zalando_Germany_Dataset/dresses/images/e4e_images/test/*.jpg")
test_skus = [elem.split('/')[-1] for elem in test_skus]
print("Training Set:")
display(np.mean(list({k:v for k,v in lpips_scores_00003.items() if k in train_skus}.values())))
print("Test Set:")
display(np.mean(list({k:v for k,v in lpips_scores_00003.items() if k in test_skus}.values())))

Training Set:


0.14491837734383903

Test Set:


0.14389682245330823

### Calculate LPIPS Scores for e4e from 00005_snapshot_1200

In [ ]:
lpips_scores_00005 = {}
for sku in tqdm(skus):
    score = get_lpips(sku, generated_path_00005, real_path)
    lpips_scores_00005[sku] = score

with open(scores_save_path_00005, 'wb') as handle:
    pickle.dump(lpips_scores_00005, handle, protocol=pickle.HIGHEST_PROTOCOL)

print(f"Mean LPIPS scores: {np.mean(list(lpips_scores_00005.values()))}")